## Imporerer pakker

In [262]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.compose import ColumnTransformer

# Model Selection and Evaluation
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import root_mean_squared_error, r2_score, accuracy_score, f1_score, log_loss, confusion_matrix

# Regression Models
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, HistGradientBoostingRegressor, AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR



# Classification Models
from sklearn.linear_model import LogisticRegression, HuberRegressor, BayesianRidge, LassoLars, PassiveAggressiveRegressor, TheilSenRegressor, SGDRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

# Other Model Utilities
from sklearn.dummy import DummyRegressor
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.tree import plot_tree

from sklearn.metrics import make_scorer
from sklearn.preprocessing import PolynomialFeatures, StandardScaler

## Laster inn filer

In [263]:
train_df = pd.read_csv('train_df.csv')
val_df = pd.read_csv('validation_df.csv')
test_df = pd.read_csv('test_df.csv')

pd.set_option('display.max_columns', None)
pd.reset_option('display.float_format')

## Ser på manglende verdier

In [264]:
# Get the count of missing values in each column
missing_columns = train_df.isna().sum()
columns_with_no_missing = missing_columns[missing_columns == 0].index.tolist()

# Convert the series to a DataFrame and transpose it
missing_columns_df = pd.DataFrame(missing_columns, columns=['Manglende verdier'])
missing_columns_df = missing_columns_df.drop(columns_with_no_missing)

# Display the DataFrame
missing_columns_df

,Manglende verdier
alder,2
utdanning,955
inntekt,2151
etnisitet,26
blod_ph,1365
glukose,2683
lege_overlevelsesestimat_2mnd,984
lege_overlevelsesestimat_6mnd,974
dnr_status,4655
adl,1218


In [265]:
train_df


,oppholdslengde,alder,kjønn,utdanning,inntekt,etnisitet,sykehusdød,blodtrykk,hvite_blodlegemer,hjertefrekvens,respirasjonsfrekvens,kroppstemperatur,lungefunksjon,serumalbumin,bilirubin,kreatinin,natrium,blod_ph,glukose,blodurea_nitrogen,urinmengde,sykdomskategori_id,sykdomskategori,dødsfall,sykdom_underkategori,antall_komorbiditeter,koma_score,fysiologisk_score,apache_fysiologisk_score,overlevelsesestimat_2mnd,overlevelsesestimat_6mnd,diabetes,demens,kreft,lege_overlevelsesestimat_2mnd,lege_overlevelsesestimat_6mnd,dnr_status,adl
0,63,65.0,male,NaN,35.0,white,0,72.0,17.199219,125.0,25.0,36.50000,330.00000,1.899902,2.899902,1.799805,149.0,7.419922,108.0,30.00,1550.0,A1s,ARF/MOSF,0,ARF/MOSF w/Sepsis,0,0.0,26.000000,44.0,0.706909,0.617920,0,0,no,0.80,0.8,NaN,0.0
1,5,66.0,female,NaN,NaN,white,0,61.0,8.398438,60.0,30.0,37.00000,217.12500,1.899902,0.699951,0.500000,139.0,7.489258,NaN,6.51,2502.0,A1s,ARF/MOSF,0,ARF/MOSF w/Sepsis,0,26.0,18.000000,31.0,0.823975,0.763916,0,0,no,0.90,0.9,NaN,NaN
2,19,64.0,female,14.0,NaN,white,0,102.0,6.000000,64.0,24.0,35.89844,333.30000,3.399902,0.399963,0.699951,137.0,NaN,NaN,6.51,2502.0,ChE,Cancer,1,Lung Cancer,2,0.0,18.296875,15.0,0.684937,0.391968,0,0,metastatic,0.90,0.6,NaN,1.0
3,4,64.0,female,14.0,8.0,white,0,76.0,9.000000,93.0,16.0,36.29688,333.30000,3.500000,1.010000,1.010000,140.0,NaN,NaN,6.51,2502.0,ChE,Cancer,1,Colon Cancer,1,9.0,15.599609,14.0,0.816895,0.605957,0,0,metastatic,0.80,0.3,NaN,0.0
4,19,64.0,male,12.0,20.0,white,0,60.0,11.798828,120.0,30.0,35.59375,172.50000,3.500000,21.796875,8.898438,139.0,7.369141,168.0,83.00,9.0,A1s,ARF/MOSF,1,ARF/MOSF w/Sepsis,1,9.0,35.000000,44.0,0.482971,0.364990,0,0,no,0.40,0.3,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5408,7,51.0,male,0.0,8.0,white,0,103.0,7.799805,66.0,26.0,36.69531,333.30000,3.500000,1.010000,0.699951,139.0,NaN,237.0,24.00,1800.0,A1s,ARF/MOSF,0,MOSF w/Malig,2,0.0,26.597656,34.0,0.613892,0.507935,0,0,yes,NaN,NaN,NaN,6.0
5409,7,36.0,female,9.0,8.0,black,1,49.0,6.699219,130.0,10.0,36.89844,380.00000,3.500000,3.799805,3.299805,143.0,7.419922,NaN,6.51,2502.0,A1s,ARF/MOSF,1,ARF/MOSF w/Sepsis,0,44.0,36.093750,49.0,0.539917,0.425964,0,0,no,NaN,NaN,NaN,1.0
5410,26,78.0,male,NaN,NaN,white,1,117.0,16.898438,70.0,20.0,35.59375,169.56250,3.599609,0.399963,1.699951,146.0,7.319336,NaN,6.51,2502.0,BrY,COPD/CHF/Cirrhosis,1,COPD,1,9.0,23.597656,40.0,0.705933,0.544922,0,0,no,0.50,0.4,NaN,NaN
5411,7,62.0,female,NaN,NaN,white,1,51.0,9.898438,126.0,20.0,39.09375,85.70312,2.500000,1.599854,0.699951,149.0,7.369141,404.0,11.00,6020.0,BrY,COPD/CHF/Cirrhosis,1,COPD,3,0.0,25.296875,17.0,0.723999,0.568970,0,0,metastatic,0.00,0.0,dnr ved innleggelse,NaN


## Variabelutvinning

In [266]:
# forberede y 
def prepare_y(df):
    return df.oppholdslengde.values
def prepare_ys(dfs):
    return (prepare_y(df) for df in dfs)

In [267]:
def prepare_X(df, onehot_encoder=None):
    # List of numeric columns to include in the dataset
    numeric_cols = [
        'alder', 'utdanning', 'blodtrykk', 'hvite_blodlegemer', 'hjertefrekvens', 
        'respirasjonsfrekvens', 'kroppstemperatur', 'lungefunksjon', 'serumalbumin', 
        'bilirubin', 'kreatinin', 'natrium', 'blod_ph', 'glukose', 'blodurea_nitrogen', 
        'urinmengde', 'antall_komorbiditeter', 'koma_score', 'fysiologisk_score', 
        'apache_fysiologisk_score', 'overlevelsesestimat_2mnd', 'overlevelsesestimat_6mnd', 
        'lege_overlevelsesestimat_2mnd', 'lege_overlevelsesestimat_6mnd', 'adl'
    ]

    # List of categorical columns to be encoded
    categorical_cols = ['kjønn', 'etnisitet', 'sykdomskategori_id', 'sykdomskategori', 
                        'sykdom_underkategori', 'dnr_status', 'kreft']

    # Step 1: One-hot encode the specified categorical columns using OneHotEncoder
    if onehot_encoder is None:
        onehot_encoder = OneHotEncoder(drop=None, sparse_output=False)
        df_encoded = onehot_encoder.fit_transform(df[categorical_cols])
    else:
        df_encoded = onehot_encoder.transform(df[categorical_cols])

    # Convert the encoded data to a DataFrame with appropriate column names
    encoded_col_names = onehot_encoder.get_feature_names_out(categorical_cols)
    df_encoded = pd.DataFrame(df_encoded, columns=encoded_col_names, index=df.index)

    # Step 2: Handle the logic to set kreft_yes if kreft_metastatic is true
    if 'kreft_metastatic' in df_encoded.columns:
        df_encoded['kreft_yes'] = df_encoded.get('kreft_yes', 0).astype(bool) | df_encoded['kreft_metastatic'].astype(bool)
    
    # Step 3: Create the 'sykdom_Other_Cancer' variable
    df_encoded['sykdom_underkategori_Other Cancer'] = (
        df_encoded['kreft_yes'].astype(bool) &  # Must have some type of cancer
        ~df_encoded.get('sykdom_underkategori_Colon Cancer', False).astype(bool) &  # Not colon cancer
        ~df_encoded.get('sykdom_underkategori_Lung Cancer', False).astype(bool)  # Not lung cancer
    )

    df_encoded.drop(columns=['kreft_no'], inplace=True)
    df_encoded.drop(columns=['kjønn_female'], inplace=True)

    # Step 3: Create new severity level indicators based on 'fysiologisk_score'
    df['alvorlighetsgrad_høy'] = (df['fysiologisk_score'] > 60).astype(bool)
    df['alvorlighetsgrad_lav'] = (df['fysiologisk_score'] < 10).astype(bool)

    # Additional columns generated from transformations
    additional_cols = ['alvorlighetsgrad_høy', 'alvorlighetsgrad_lav']

    # Concatenate the numeric data, encoded categorical data, and additional columns into the final DataFrame
    df_final = pd.concat([df[numeric_cols], df_encoded, df[additional_cols]], axis=1)

    return df_final, onehot_encoder

# Function to prepare multiple DataFrames using the same encoder
def prepare_Xs(dfs):
    onehot_encoder = None  # Initialize the encoder variable outside the loop
    transformed_dfs = []

    for df in dfs:
        df_transformed, onehot_encoder = prepare_X(df, onehot_encoder)
        transformed_dfs.append(df_transformed)

    return transformed_dfs  # Return the list of transformed DataFrames

In [268]:
# faktiske dataforberedelse
X_train, X_val, X_test = prepare_Xs((train_df, val_df, test_df))
y_train, y_val, y_test = prepare_ys((train_df, val_df, test_df))

In [269]:
X_train

,alder,utdanning,blodtrykk,hvite_blodlegemer,hjertefrekvens,respirasjonsfrekvens,kroppstemperatur,lungefunksjon,serumalbumin,bilirubin,kreatinin,natrium,blod_ph,glukose,blodurea_nitrogen,urinmengde,antall_komorbiditeter,koma_score,fysiologisk_score,apache_fysiologisk_score,overlevelsesestimat_2mnd,overlevelsesestimat_6mnd,lege_overlevelsesestimat_2mnd,lege_overlevelsesestimat_6mnd,adl,kjønn_male,etnisitet_asian,etnisitet_black,etnisitet_hispanic,etnisitet_other,etnisitet_white,etnisitet_nan,sykdomskategori_id_A1s,sykdomskategori_id_BrY,sykdomskategori_id_ChE,sykdomskategori_id_DWw,sykdomskategori_ARF/MOSF,sykdomskategori_COPD/CHF/Cirrhosis,sykdomskategori_Cancer,sykdomskategori_Coma,sykdom_underkategori_ARF/MOSF w/Sepsis,sykdom_underkategori_CHF,sykdom_underkategori_COPD,sykdom_underkategori_Cirrhosis,sykdom_underkategori_Colon Cancer,sykdom_underkategori_Coma,sykdom_underkategori_Lung Cancer,sykdom_underkategori_MOSF w/Malig,dnr_status_dnr før innleggelse,dnr_status_dnr ved innleggelse,dnr_status_nan,kreft_metastatic,kreft_yes,sykdom_underkategori_Other Cancer,alvorlighetsgrad_høy,alvorlighetsgrad_lav
0,65.0,NaN,72.0,17.199219,125.0,25.0,36.50000,330.00000,1.899902,2.899902,1.799805,149.0,7.419922,108.0,30.00,1550.0,0,0.0,26.000000,44.0,0.706909,0.617920,0.80,0.8,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,False,False,False,False
1,66.0,NaN,61.0,8.398438,60.0,30.0,37.00000,217.12500,1.899902,0.699951,0.500000,139.0,7.489258,NaN,6.51,2502.0,0,26.0,18.000000,31.0,0.823975,0.763916,0.90,0.9,NaN,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,False,False,False,False
2,64.0,14.0,102.0,6.000000,64.0,24.0,35.89844,333.30000,3.399902,0.399963,0.699951,137.0,NaN,NaN,6.51,2502.0,2,0.0,18.296875,15.0,0.684937,0.391968,0.90,0.6,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,True,False,False,False
3,64.0,14.0,76.0,9.000000,93.0,16.0,36.29688,333.30000,3.500000,1.010000,1.010000,140.0,NaN,NaN,6.51,2502.0,1,9.0,15.599609,14.0,0.816895,0.605957,0.80,0.3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,True,False,False,False
4,64.0,12.0,60.0,11.798828,120.0,30.0,35.59375,172.50000,3.500000,21.796875,8.898438,139.0,7.369141,168.0,83.00,9.0,1,9.0,35.000000,44.0,0.482971,0.364990,0.40,0.3,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5408,51.0,0.0,103.0,7.799805,66.0,26.0,36.69531,333.30000,3.500000,1.010000,0.699951,139.0,NaN,237.0,24.00,1800.0,2,0.0,26.597656,34.0,0.613892,0.507935,NaN,NaN,6.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,True,True,False,False
5409,36.0,9.0,49.0,6.699219,130.0,10.0,36.89844,380.00000,3.500000,3.799805,3.299805,143.0,7.419922,NaN,6.51,2502.0,0,44.0,36.093750,49.0,0.539917,0.425964,NaN,NaN,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,False,False,False,False
5410,78.0,NaN,117.0,16.898438,70.0,20.0,35.59375,169.56250,3.599609,0.399963,1.699951,146.0,7.319336,NaN,6.51,2502.0,1,9.0,23.597656,40.0,0.705933,0.544922,0.50,0.4,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,False,False,False,False
5411,62.0,NaN,51.0,9.898438,126.0,20.0,39.09375,85.70312,2.500000,1.599854,0.699951,149.0,7.369141,404.0,11.00,6020.0,3,0.0,25.296875,17.0,0.723999,0.568970,0.00,0.0,NaN,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1

In [270]:
missing_columns = X_train.isna().sum()
columns_with_no_missing = missing_columns[missing_columns == 0].index.tolist()

# Convert the series to a DataFrame and transpose it
missing_columns_df = pd.DataFrame(missing_columns, columns=['Manglende verdier'])
missing_columns_df = missing_columns_df.drop(columns_with_no_missing)

# Display the DataFrame
missing_columns_df

,Manglende verdier
alder,2
utdanning,955
blod_ph,1365
glukose,2683
lege_overlevelsesestimat_2mnd,984
lege_overlevelsesestimat_6mnd,974
adl,1218


## Grunnlinjemodell

In [271]:
baseline = DummyRegressor(strategy='mean')
baseline.fit(X_train, y_train)
prediction = baseline.predict(X_val)
rmse_baseline = root_mean_squared_error(y_val, prediction)
rmse_baseline

21.59611100410102

In [272]:
baseline2 = DummyRegressor(strategy='median')
baseline2.fit(X_train, y_train)
prediction = baseline2.predict(X_val)
rmse_baseline = root_mean_squared_error(y_val, prediction)
rmse_baseline

22.712103715253104

## Modell

In [273]:
# Define imputers and models
strategies = {
    "mean": SimpleImputer(missing_values=np.nan, strategy='mean'),
    "median": SimpleImputer(missing_values=np.nan, strategy='most_frequent'),
    "removing": SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0),
    "KNN5": KNNImputer(n_neighbors=5),
    "KNN20": KNNImputer(n_neighbors=20),
    "KNN40": KNNImputer(n_neighbors=40),
    "KNN50": KNNImputer(n_neighbors=50),
    "KNN60": KNNImputer(n_neighbors=60),
    "KNN70": KNNImputer(n_neighbors=70),
    "KNN79": KNNImputer(n_neighbors=79),
    "KNN80": KNNImputer(n_neighbors=80),
    "KNN90": KNNImputer(n_neighbors=90)

}

models = {
    "RandomForest": RandomForestRegressor(n_estimators=100, min_samples_split=8, min_samples_leaf=4, max_features=3, max_depth=110, bootstrap=True, random_state=42),
    "LinearRegression": LinearRegression(),
    "Ridge": Ridge(alpha=1.0, random_state=42),
    "Lasso": Lasso(alpha=0.1, random_state=42),
    "GradientBoosting": GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42),
    "ExtraTreesRegressor" : ExtraTreesRegressor(random_state=42),
    "DecisionTreeRegressor" : DecisionTreeRegressor(random_state=42),
    "ElasticNet": ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=42),
    "AdaBoost": AdaBoostRegressor(n_estimators=100, learning_rate=0.1, random_state=42),
    "SVR": SVR(kernel='rbf', C=1.0, epsilon=0.1),
    "HuberRegressor": HuberRegressor(max_iter=10000),
    "BayesianRidge": BayesianRidge(),
    "LassoLars": LassoLars(alpha=0.1),
    "PassiveAggressiveRegressor": PassiveAggressiveRegressor(max_iter=1000, random_state=42),
    "TheilSenRegressor": TheilSenRegressor(random_state=42),
    "SGDRegressor": SGDRegressor(max_iter=1000, tol=1e-3, random_state=42)
}

val_scores = {}

# Loop through each imputer and model combination
for strategy_name, imputer in strategies.items():
    X_train_imp = imputer.fit_transform(X_train)
    X_val_imp = imputer.transform(X_val)

    for model_name, model in models.items():
        full_model_name = f"{strategy_name}_{model_name}"
        model.fit(X_train_imp, y_train)

        y_pred = model.predict(X_val_imp)
        val_scores[full_model_name] = root_mean_squared_error(y_val, y_pred)  # RMSE

# Create a DataFrame to display the results
val_scores_df = pd.DataFrame(list(val_scores.items()), columns=['Strategy_Model', 'RMSE'])

# Find the best model based on the lowest RMSE
beste_model_name = val_scores_df.loc[val_scores_df['RMSE'].idxmin(), 'Strategy_Model']
best_score = val_scores_df.loc[val_scores_df['RMSE'].idxmin(), 'RMSE']

print(f"Den beste og valgte modellen er {beste_model_name} med en RMSE på {best_score:.3f} på valideringsdata")

/Users/jakobberg/anaconda3/envs/INF161/lib/python3.12/site-packages/sklearn/linear_model/_least_angle.py:725: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 22 iterations, i.e. alpha=6.620e-01, with an active set of 22 regressors, and the smallest cholesky pivot element being 2.220e-16. Reduce max_iter or increase eps parameters.
  warnings.warn(
/Users/jakobberg/anaconda3/envs/INF161/lib/python3.12/site-packages/sklearn/linear_model/_least_angle.py:725: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 24 iterations, i.e. alpha=4.277e-01, with an active set of 24 regressors, and the smallest cholesky pivot element being 2.220e-16. Reduce max_iter or increase eps parameters.
  warnings.warn(
/Users/jakobberg/anaconda3/envs/INF161/lib/python3.12/site-packages/sklearn/linear_model/_least_angle.py:725: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 25 iterations, i.e. alpha=2.880

Den beste og valgte modellen er KNN5_RandomForest med en RMSE på 20.173 på valideringsdata


## Oversikt over ulike modeller med forskjellig strategi for imputering

In [274]:
val_scores_df = val_scores_df.sort_values('RMSE')
pd.options.display.float_format = '{:.2f}'.format
val_scores_df.head(20)

,Strategy_Model,RMSE
48,KNN5_RandomForest,20.17
144,KNN79_RandomForest,20.20
96,KNN50_RandomForest,20.20
160,KNN80_RandomForest,20.21
128,KNN70_RandomForest,20.22
176,KNN90_RandomForest,20.22
0,mean_RandomForest,20.22
32,removing_RandomForest,20.23
112,KNN60_RandomForest,20.24
16,median_RandomForest,20.24


In [282]:
# Define the imputers with different values of n_neighbors for KNN
imputers = {f"KNNImputer_n={n}": KNNImputer(n_neighbors=n) for n in range(1, 100, 1)}

# Define the model(s) you want to test
models = {
    "RandomForest": RandomForestRegressor(n_estimators=100, min_samples_split=8, min_samples_leaf=4, 
                                          max_features=3, max_depth=110, bootstrap=True, random_state=42),
}

val_scores = {}

# Loop through each imputer and model combination
for strategy_name, imputer in imputers.items():
    X_train_imp = imputer.fit_transform(X_train)
    X_val_imp = imputer.transform(X_val)

    for model_name, model in models.items():
        full_model_name = f"{strategy_name}_{model_name}"
        model.fit(X_train_imp, y_train)

        y_pred = model.predict(X_val_imp)
        rmse = root_mean_squared_error(y_val, y_pred)  # Calculate RMSE
        val_scores[full_model_name] = rmse  # Store the RMSE for each combination

# Create a DataFrame to display the results
val_scores_df = pd.DataFrame(list(val_scores.items()), columns=['Strategy_Model', 'RMSE'])

# Find the best model based on the lowest RMSE
best_model_name = val_scores_df.loc[val_scores_df['RMSE'].idxmin(), 'Strategy_Model']
best_score = val_scores_df.loc[val_scores_df['RMSE'].idxmin(), 'RMSE']

print(f"The best model is {best_model_name} with an RMSE of {best_score:.3f} on the validation data")


The best model is KNNImputer_n=93_RandomForest with an RMSE of 20.134 on the validation data


## Trener forskjellige modeller med hyperparamtere

In [286]:
# Step 1: Create the pipeline with KNNImputer and RandomForestRegressor
pipeline = Pipeline([
    ('imputer', KNNImputer(n_neighbors=93)),  
    ('model', RandomForestRegressor(random_state=42))  
])

# Step 2: Set up the parameter grid for the RandomForestRegressor
param_grid = {
    'model__n_estimators': [100, 200, 300, 1000],  # Number of trees in the forest
    'model__max_depth': [10, 50, 100, None],  # Maximum depth of each tree
    'model__max_features': ['auto', 'sqrt'],  # Number of features to consider for each split
    'model__min_samples_split': [2, 5, 10],  # Minimum number of samples required to split a node
    'model__min_samples_leaf': [1, 2, 4],  # Minimum number of samples required at each leaf node
    'model__bootstrap': [True, False]  # Whether bootstrap samples are used when building trees
}

# Step 3: Set up GridSearchCV with the pipeline
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, 
                           cv=3, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')

# Step 4: Fit the grid search to the training data
grid_search.fit(X_train, y_train)

# Step 5: Get the best parameters and best model from the grid search
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Step 6: Calculate RMSE on the validation set using the best model
y_pred = best_model.predict(X_val)
best_rmse = root_mean_squared_error(y_val, y_pred)  # Calculate RMSE
print(f"Best parameters found: {best_params}")
print(f"Best model: {best_model}")
print(f"Best RMSE on validation set: {best_rmse:.3f}")

Fitting 3 folds for each of 576 candidates, totalling 1728 fits
[CV] END model__bootstrap=True, model__max_depth=10, model__max_features=auto, model__min_samples_leaf=1, model__min_samples_split=2, model__n_estimators=200; total time=   4.1s
[CV] END model__bootstrap=True, model__max_depth=10, model__max_features=auto, model__min_samples_leaf=1, model__min_samples_split=2, model__n_estimators=100; total time=   4.2s
[CV] END model__bootstrap=True, model__max_depth=10, model__max_features=auto, model__min_samples_leaf=1, model__min_samples_split=2, model__n_estimators=300; total time=   4.3s
[CV] END model__bootstrap=True, model__max_depth=10, model__max_features=auto, model__min_samples_leaf=1, model__min_samples_split=2, model__n_estimators=300; total time=   4.5s
[CV] END model__bootstrap=True, model__max_depth=10, model__max_features=auto, model__min_samples_leaf=1, model__min_samples_split=2, model__n_estimators=200; total time=   5.4s
[CV] END model__bootstrap=True, model__max_dep

/Users/jakobberg/anaconda3/envs/INF161/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
864 fits failed out of a total of 1728.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
329 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/jakobberg/anaconda3/envs/INF161/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/jakobberg/anaconda3/envs/INF161/lib/python3.12/site-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jakobberg/anaconda3/envs/

Best parameters found: {'model__bootstrap': True, 'model__max_depth': 50, 'model__max_features': 'sqrt', 'model__min_samples_leaf': 4, 'model__min_samples_split': 2, 'model__n_estimators': 1000}
Best model: Pipeline(steps=[('imputer', KNNImputer(n_neighbors=93)),
                ('model',
                 RandomForestRegressor(max_depth=50, max_features='sqrt',
                                       min_samples_leaf=4, n_estimators=1000,
                                       random_state=42))])
Best RMSE on validation set: 20.294


In [ ]:
# Final evaluation on test set
test_predictions = best_model.predict(X_test)
test_rmse = root_mean_squared_error(y_test, test_predictions)


test_r2 = r2_score(y_test, test_predictions)
print(f'Test RMSE: {test_rmse}')
print(f'Test R-squared: {test_r2}')

# Feature importance
feature_importances = pd.Series(best_model.feature_importances_, index=X_train.columns)
feature_importances.sort_values(ascending=False, inplace=True)
print(feature_importances.head(10))
